In [1]:
import geopandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import laspy
from shapely.geometry import box, Polygon, MultiPolygon, GeometryCollection
import uuid
import scipy.stats
import seaborn as sns
import math

In [2]:
inventory: pd.DataFrame = pd.read_csv(
    "C:/Users/joaov/Documents/UFMG/TCC/Dataset/DUC_A01_2016_inventory.csv",
    encoding="ISO-8859-1",
)

In [3]:
inventory_plot_location: geopandas.GeoDataFrame = geopandas.read_file(
    "C:/Users/joaov/Documents/UFMG/TCC/Dataset/DUC_A01_2016_PLOTLOCATION/duc_a01_2016_plotlocation.shx",
)

In [4]:
inventory.head()

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,20160824,840126.4377,9674180.705
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,20160824,840109.4829,9674171.803
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,20160824,840125.6847,9674181.313


In [5]:
inventory["scientific.name"].unique(), len(inventory["scientific.name"].unique())

(array(['Licania guianensis', 'Sacoglottis guianensis', 'Pourouma minor',
        'Pouteria gongrijpii', 'Peltogyne lecointei', nan,
        'Protium paniculatum', 'Qualea paraensis', 'Mouriri collocarpa',
        'Eschweilera ovata', 'Pouteria sp.', 'Ormosia grossa',
        'Ocotea cernua', 'Eperua bijuga', 'Brosimum rubescens',
        'Oenocarpus bataua', 'Hevea brasiliensis', 'Inga thibaudiana',
        'Eriotheca globosa', 'Iryanthera paraensis', 'Dipteryx odorata',
        'Sterculia pruriens', 'Miconia poeppigii', 'Sloanea guianensis',
        'Poeppigia procera', 'Xylopia sp.', 'Neea oppositifolia',
        'Geissospermum sericeum', 'Goupia glabra', 'Eschweilera micrantha',
        'Inga alba', 'Diospyros artanthifolia', 'Eschweilera coriacea',
        'Micrandra minor', 'Trichilia micrantha',
        'Tetragastris panamensis', 'Ormosia coccinea', 'Guarea guidonia',
        'Couepia robusta', 'Pouteria guianensis', 'Ocotea canaliculata',
        'Licania octandra', 'Guatteria 

In [6]:
inventory["family.name"].unique(), len(inventory["family.name"].unique())

(array(['Chrysobalanaceae', 'Humiriaceae', 'Urticaceae', 'Sapotaceae',
        'Fabaceae', nan, 'Burseraceae', 'Vochysiaceae', 'Melastomataceae',
        'Lecythidaceae', 'Lauraceae', 'Moraceae', 'Arecaceae',
        'Euphorbiaceae', 'Malvaceae', 'Myristicaceae', 'Elaeocarpaceae',
        'Annonaceae', 'Nyctaginaceae', 'Apocynaceae', 'Goupiaceae',
        'Ebenaceae', 'Meliaceae', 'Violaceae', 'Olacaceae',
        'Caryocaraceae', 'Combretaceae', 'Loganiaceae', 'Myrtaceae',
        'Simaroubaceae', 'Rubiaceae', 'Bignoniaceae', 'Malpighiaceae',
        'Salicaceae', 'Boraginaceae', 'Anacardiaceae', 'Lamiaceae',
        'Ochnaceae', 'Clusiaceae', 'Calophyllaceae'], dtype=object),
 40)

In [7]:
wood_density_dataframe = pd.read_excel(
    "C:/Users/joaov/Documents/UFMG/TCC/Dataset/GlobalWoodDensityDatabase.xls",
    sheet_name="Data",
)


wood_density_dataframe.head()

,Number,Family,Binomial,"Wood density (g/cm^3), oven dry mass/fresh volume",Region,Reference Number
0,1,Fabaceae,Abarema jupunba,0.78000,South America (tropical),114
1,2,Fabaceae,Abarema jupunba,0.66000,South America (tropical),198
2,3,Fabaceae,Abarema jupunba,0.55104,South America (tropical),52
3,4,Fabaceae,Abarema jupunba,0.53382,South America (tropical),65
4,5,Fabaceae,Abarema jupunba,0.55104,South America (tropical),189


In [8]:
wood_density_dataframe = wood_density_dataframe.rename(
    {
        "Family": "family.name",
        "Binomial": "scientific.name",
        "Wood density (g/cm^3), oven dry mass/fresh volume": "wood.density",
        "Reference Number": "reference.number",
    },
    axis=1,
)

wood_density_dataframe.head()

,Number,family.name,scientific.name,wood.density,Region,reference.number
0,1,Fabaceae,Abarema jupunba,0.78000,South America (tropical),114
1,2,Fabaceae,Abarema jupunba,0.66000,South America (tropical),198
2,3,Fabaceae,Abarema jupunba,0.55104,South America (tropical),52
3,4,Fabaceae,Abarema jupunba,0.53382,South America (tropical),65
4,5,Fabaceae,Abarema jupunba,0.55104,South America (tropical),189


In [9]:
mean_wood_density_dataframe = (
    wood_density_dataframe.groupby(["family.name", "scientific.name"])["wood.density"]
    .mean("wood.density")
    .reset_index()
)

In [10]:
mean_wood_density_dataframe = mean_wood_density_dataframe.rename(
    {
        "wood.density": "scientific.name.wood.density",
    },
    axis=1,
)

mean_wood_density_dataframe.head()

,family.name,scientific.name,scientific.name.wood.density
0,Acanthaceae,Avicennia alba,0.586667
1,Acanthaceae,Avicennia germinans,0.776424
2,Acanthaceae,Avicennia intermedia,0.640000
3,Acanthaceae,Avicennia marina,0.647663
4,Acanthaceae,Avicennia nitida,0.900000


In [11]:
inventory.loc[inventory["common.name"] == "faveira", "scientific.name"] = (
    "Parkia platycephala"
)

inventory[inventory["common.name"] == "faveira"].head()

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing
159,DUC_A01,5,020a,faveira,Parkia platycephala,Fabaceae,16.5,O,False,NaN,20160830,841259.5417,9674444.634


In [12]:
inventory = inventory.merge(
    mean_wood_density_dataframe,
    on="scientific.name",
    how="left",
)


# wood_density_dataframe["reference.number"].apply(str)


inventory.head()

,area,plot,tree,common.name,scientific.name,family.name_x,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing,family.name_y,scientific.name.wood.density
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,NaN,NaN
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868,Humiriaceae,0.835775
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,20160824,840126.4377,9674180.705,Urticaceae,0.435953
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,20160824,840109.4829,9674171.803,Sapotaceae,0.799403
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,20160824,840125.6847,9674181.313,Fabaceae,0.763333


In [13]:
inventory = inventory.rename(
    {
        "family.name_x": "family.name",
        "Binomial": "scientific.name",
        "Wood density (g/cm^3), oven dry mass/fresh volume": "wood.density",
        "Reference Number": "reference.number",
    },
    axis=1,
)

inventory.head()

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing,family.name_y,scientific.name.wood.density
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,NaN,NaN
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868,Humiriaceae,0.835775
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,20160824,840126.4377,9674180.705,Urticaceae,0.435953
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,20160824,840109.4829,9674171.803,Sapotaceae,0.799403
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,20160824,840125.6847,9674181.313,Fabaceae,0.763333


In [14]:
mean_wood_density_dataframe = (
    wood_density_dataframe.groupby("family.name")["wood.density"].mean().reset_index()
)

In [15]:
mean_wood_density_dataframe = mean_wood_density_dataframe.rename(
    {
        "wood.density": "family.wood.density",
    },
    axis=1,
)

mean_wood_density_dataframe.head()

,family.name,family.wood.density
0,Acanthaceae,0.670709
1,Achariaceae,0.605300
2,Actinidiaceae,0.398541
3,Adoxaceae,0.446333
4,Akaniaceae,0.560710


In [16]:
mean_wood_density_dataframe.head()

,family.name,family.wood.density
0,Acanthaceae,0.670709
1,Achariaceae,0.605300
2,Actinidiaceae,0.398541
3,Adoxaceae,0.446333
4,Akaniaceae,0.560710


In [17]:
inventory.head()

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing,family.name_y,scientific.name.wood.density
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,NaN,NaN
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868,Humiriaceae,0.835775
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,20160824,840126.4377,9674180.705,Urticaceae,0.435953
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,20160824,840109.4829,9674171.803,Sapotaceae,0.799403
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,20160824,840125.6847,9674181.313,Fabaceae,0.763333


In [18]:
inventory = inventory.merge(mean_wood_density_dataframe, on="family.name", how="left")

In [19]:
inventory.head()

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing,family.name_y,scientific.name.wood.density,family.wood.density
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,NaN,NaN,0.780701
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868,Humiriaceae,0.835775,0.771500
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,20160824,840126.4377,9674180.705,Urticaceae,0.435953,0.344090
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,20160824,840109.4829,9674171.803,Sapotaceae,0.799403,0.702085
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,20160824,840125.6847,9674181.313,Fabaceae,0.763333,0.678134


In [20]:
inventory["wood.density"] = inventory["scientific.name.wood.density"].combine_first(
    inventory["family.wood.density"]
)

In [21]:
inventory

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing,family.name_y,scientific.name.wood.density,family.wood.density,wood.density
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,NaN,NaN,0.780701,0.780701
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868,Humiriaceae,0.835775,0.771500,0.835775
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,20160824,840126.4377,9674180.705,Urticaceae,0.435953,0.344090,0.435953
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,20160824,840109.4829,9674171.803,Sapotaceae,0.799403,0.702085,0.799403
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,20160824,840125.6847,9674181.313,Fabaceae,0.763333,0.678134,0.763333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164,DUC_A01,20,048a,quarubarana,Erisma uncinatum,Vochysiaceae,24.0,O,False,NaN,20160830,840072.5767,9674390.969,Vochysiaceae,0.523047,0.551651,0.523047
1165,DUC_A01,20,049a,fava saboeiro,Abarema jupunba,Fabaceae,14.0,O,False,NaN,20160830,840071.0245,9674395.507,Fabaceae,0.585129,0.678134,0.585129
1166,DUC_A01,20,050a,mamorana,Eriotheca globosa,Malvaceae,22.0,O,False,NaN,20160830,840069.0191,9674396.896,Malvaceae,0.410000,0.476090,0.410000
1167,DUC_A01,20,051a,louro cheiroso,Diospyros artanthifolia,Ebenaceae,11.9,O,False,NaN,20160830,840069.1150,9674398.094,NaN,NaN,0.697848,0.697848


In [25]:
inventory[inventory["wood.density"].isnull()]["common.name"].unique()

array(['morta', 'tamaguare', 'ucuuba da varzea', 'carana', 'fava rosa',
       'pincel de macaco', 'breu brrote', 'abacabeira', 'jarana vermelha',
       'taguari', 'tanenbuco amarelo', 'acaizeiro', 'liana',
       'faveira amargosa', 'zaguari', 'virola/ucuuba preta',
       'muiracatiara rajada', 'abiu rosadinha', 'burra leteira',
       'abirana', 'gaivota', 'abirana branca', 'carape', 'mirindiba',
       'pedra ume', 'brteu branco', 'timbora', 'mamominho',
       'ucuuba folha grande', 'tanenbuco', 'faveira rosa',
       'nao identificada', 'guajara vermelho', 'muiracacaco',
       'acariguara', 'acariguarana', 'puriu', ' liana', 'seringuarana',
       'nao indetificada', 'manguirana', 'virola(ucuuba)', 'sorvao',
       'urucurana branca', 'tento cachua', 'ipê amarelo', 'uxi coroa',
       'quariuba', 'mirapiranga', 'ucuuba t.f', 'envira cunacaru',
       'seringarana preta', 'pocoro', 'cacaui', 'urucurana branco'],
      dtype=object)

In [27]:
inventory[inventory["wood.density"].isnull()]["family.name"].unique()

array([nan], dtype=object)

In [28]:
inventory[inventory["wood.density"].isnull()].count()

area                            183
plot                            183
tree                            183
common.name                     183
scientific.name                   0
family.name                       0
DBH                             182
type                            183
Dead                            183
D.class                          74
date                            183
UTM.Easting                     183
UTM.Northing                    183
family.name_y                     0
scientific.name.wood.density      0
family.wood.density               0
wood.density                      0
dtype: int64

In [32]:
inventory = inventory.dropna(subset=["wood.density"]).drop(
    ["scientific.name.wood.density", "family.wood.density", "family.name_y"], axis=1
)

In [48]:
inventory["tree.height"] = 38.1 * (1 - np.exp(-0.0693 * inventory["DBH"] ** 0.826))

In [49]:
inventory

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing,wood.density,tree.height
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,0.780701,31.430128
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868,0.835775,30.146223
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,20160824,840126.4377,9674180.705,0.435953,26.597750
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,20160824,840109.4829,9674171.803,0.799403,29.025875
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,20160824,840125.6847,9674181.313,0.763333,14.149619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164,DUC_A01,20,048a,quarubarana,Erisma uncinatum,Vochysiaceae,24.0,O,False,NaN,20160830,840072.5767,9674390.969,0.523047,23.463946
1165,DUC_A01,20,049a,fava saboeiro,Abarema jupunba,Fabaceae,14.0,O,False,NaN,20160830,840071.0245,9674395.507,0.585129,17.459601
1166,DUC_A01,20,050a,mamorana,Eriotheca globosa,Malvaceae,22.0,O,False,NaN,20160830,840069.0191,9674396.896,0.410000,22.459927
1167,DUC_A01,20,051a,louro cheiroso,Diospyros artanthifolia,Ebenaceae,11.9,O,False,NaN,20160830,840069.1150,9674398.094,0.697848,15.807484
